<hr style="border:2px solid">
<h1 style="text-align:center; font-size: 3em;">Python Game Development</h1>
<hr style="border-top:1px dashed">

### Snake Game
#### Project Requirements:

- README File
- Flowchart
- Game code
- Trello Board

Both the README file and the flowchart are in the submited folder.
The game code is in this Jupyter Notebook.
Trello board access:https://trello.com/invite/b/cNniUI2R/ATTI4823f61a4c7ccbf909430ee01fbec274735431C7/python-game-development

<hr style="border:2px solid">
<h1 style="text-align:center; font-size: 2em;">Python Code</h1>
<hr style="border-top:1px dashed">

In [1]:
# Import necessary libraries 
import pygame # To create the game display
import random # To randomize the position where the snake and fruit start the game

# Initialize Pygame 
pygame.init()
pygame.mixer.init()

# Set up the screen
screen_width = 600
screen_height = 400
# the screen width and height are created as variable to facilite editing them
# for example if we want to test the game in a smaller/ larger display
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Snake Game") # Labels the top part of the window of the game

# Load sounds
# These sounds were downloaded from pixabay.com and added to the submission folder
eat_sound = pygame.mixer.Sound("eatingSound.mp3") # mixer allows to load and play sounds
game_over_sound = pygame.mixer.Sound("endGame.mp3")


# Load background music
# Put on replay
pygame.mixer.music.load("backgroundSound.mp3")
pygame.mixer.music.play(-1)  # Play music in an infinite loop

# Colors
# Defining colours in the beginning facilitates editing the display colours
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)
green = (0, 255, 0)

# The following line of code is introduced to control the frame rate of the game
clock = pygame.time.Clock()

font_style = pygame.font.SysFont(None, 30)

def message(msg, color, y_displacement=0):
    '''
        This functions displays a message on screen with a given color and displacement.
    '''
    # Renders the message (string introduced when we call the function) with a specific color
    mesg = font_style.render(msg, True, color) 
    # Retrieves the text and places it according to the y displacement introduced and the screen height and width
    text_rect = mesg.get_rect(center=(screen_width / 2, screen_height / 2 + y_displacement))
    # Draws the message on the main screen
    screen.blit(mesg, text_rect)

    
def game_menu():
    
    """
        This function is used to display the game menu with difficulty levels
        It allows the user to press a number 0 to 5 to select difficulty level.
        Returns selected snake_speed and block_size (these are the variables used to
        control the difficulty levels).
    """
    # Menu will display before the difficulty level is selected

    menu = True 

    while menu:
        screen.fill(white)
        message("Welcome to Snake Game", green, -150) # The number at the end shows the position of the text in the pop up screen
        message("Select Difficulty:", black, -80) 
        message("0: Super Easy", black, -40)
        message("1: Easy", black, 0)
        message("2: Medium", black, 40)
        message("3: Hard", black, 80)
        message("4: Super Hard", black, 120)
        message("5: Impossible", black, 160)
        
        pygame.display.update() # Updates the entire display

        # This for loop allows the menu to disapear when a number from 0 to 5 is selected
        # This also attributes a value to the variables snake_speed and block_size which will determine the difficulty of the game
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_0:
                    snake_speed =10
                    block_size = 50
                    menu = False # This closes the menu display do allow the game to start
                elif event.key == pygame.K_1:
                    snake_speed = 10
                    block_size = 30
                    menu = False
                elif event.key == pygame.K_2:
                    snake_speed = 15
                    block_size = 10
                    menu = False
                elif event.key == pygame.K_3:
                    snake_speed = 20
                    block_size = 10
                    menu = False
                elif event.key == pygame.K_4:
                    snake_speed = 40
                    block_size = 10
                    menu = False
                elif event.key == pygame.K_5:
                    snake_speed = 70
                    block_size = 10
                    menu = False
                elif event.key == pygame.K_q:
                    pygame.quit()
                    quit()

    return snake_speed, block_size


def gameLoop():

    '''
    This function represent the main loop of the game
    It keeps running while the snake hasn't bit itself or boundaries are touched
    '''
    # Defining main variables for the game
    snake_speed, block_size = game_menu()
    game_over = False
    game_close = False

    # Snake initial position and direction
    x1 = screen_width / 2
    y1 = screen_height / 2

    # the next 2 variables will allows the snake to 
    # change direction according to the arrow button clicked
    x1_change = 0 
    y1_change = 0

    # Snake body - these variables allow the calculations necessary for the snake to increase
    # in size when it eats a fruit
    snake_list = [] 
    length_of_snake = 1

    # Initializes the score each game 
    score = 0

    # Initial food position
    foodx = round(random.randrange(0, screen_width - block_size) / block_size) * block_size
    foody = round(random.randrange(0, screen_height - block_size) / block_size) * block_size
    # random.randrange(0, screen_width - block_size) - generates a random integer considering the block_size
    # so it doesnt appear outside the screen boundaries
    # / block_size - scales down the randomized coordinate so it fits withing the defined screen size
    # round(...) - rounds it up to the nearest integer
    # * block_size - scales it back to ensure the correct block_size in the defined grid
    # overall -> ensures the food item is spawn at a randomposition within the game screen while ensuring 
    # it aligns with the grid

    # while loop constantly running while the game is being played
    while not game_over: 
        
        while game_close == True: # a new display is presented when the user loses the game
            
            screen.fill(white)
            message("Game Over!", red,-70)
            message("Score: " + str(score),black ,20)
            message("Q-Quit", black,80)
            message("C-Play Again", black,120)

            pygame.display.update()
            
            # Play game over sound
            game_over_sound.play(maxtime=0)

            # When the game is over allow to either quite the game or play again by clicking 'Q' or 'C' respectively
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:
                        game_over = True
                        game_close = False
                    if event.key == pygame.K_c:
                        gameLoop() # Re starts the game loop

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True

            # Allows the arrows keys to move the snake blocks
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    x1_change = -block_size
                    y1_change = 0
                elif event.key == pygame.K_RIGHT:
                    x1_change = block_size
                    y1_change = 0
                elif event.key == pygame.K_UP:
                    y1_change = -block_size
                    x1_change = 0
                elif event.key == pygame.K_DOWN:
                    y1_change = block_size
                    x1_change = 0

        # The loop is running at every point in time the snake is moving
        # This if statement informs the programme if the snake touches a boundarie
        # according to the x and y coordenates of the head of the snake
        # And terminates the game if a boundarie (according to the height and width of the screen) is touched
        if x1 >= screen_width or x1 < 0 or y1 >= screen_height or y1 < 0:
            game_close = True

        # Changes the coordinates of the head of the snakes as the while loop is running
        # takes into account the current position of the snake block and the arrow keys pressed
        x1 += x1_change
        y1 += y1_change

        # Background of the screen continues black
        screen.fill(black)

        # Draws the food
        pygame.draw.rect(screen, green, [foodx, foody, block_size, block_size])

        # The snake head is initiated as a list to allow to increase the snakes size 
        # and to also allow the block representative of the snake to move coordinates
        # according to the arrow keys pressed
        # this increase is done via coordinates x and y
        
        # The following code updates the snake head position. 
        # it had to an empty list the new position of the snake head
        snake_head = []
        snake_head.append(x1)
        snake_head.append(y1)

        #snake_list keepts track of all the segments of the snakes body
        snake_list.append(snake_head) # appends x and y coordinates of the snake head in 1 position
        
        # The if statement allow the snake to maintain the correct length 
        if len(snake_list) > length_of_snake:
            del snake_list[0]

        # Check if snake eats the food
        # This loop check the new head position and compares it to the coordinates of all the snakes length
        # this terminates the game when the snake colides with itself
        for x in snake_list[:-1]:
            if x == snake_head:
                game_close = True

        # Draws the snake
        for segment in snake_list:
            pygame.draw.rect(screen, white, [segment[0], segment[1], block_size, block_size])

        pygame.display.update()

        # Check if snake eats the food
        # the if condiniton checks if the coordinates of the snake head are within 1 block of the food coordinates
        if (foodx - block_size <= snake_list[-1][0] <= foodx + block_size) and (foody - block_size <= snake_list[-1][1] <= foody + block_size):
            foodx = random.randrange(0, screen_width - block_size, block_size)#randomizes the position of the food when its eaten
            foody = random.randrange(0, screen_height - block_size, block_size)
            length_of_snake += 1 # increases the snake length
            score += 1
            
            eat_sound.play()  # Play eat sound

        clock.tick(snake_speed)
        # Cap the frame rate at the snake_speed frames per second

    # ends all pygame functions
    pygame.quit()
    # exists python interpreter
    quit()

# Calls the main game loop function
gameLoop()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
